# Assignment n°2

In [50]:
import numpy as np 
import pandas as pd 

from matplotlib.pyplot import subplots 
from plotnine import *

import statsmodels.api as sm

from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)

from sklearn.model_selection import train_test_split

from functools import partial
from sklearn.model_selection import ( \
    cross_validate,
    KFold,
    ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

from sklearn.utils import shuffle

from tqdm import tqdm

import warnings

In [3]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
communities_and_crime = fetch_ucirepo(id=183) 
  
# data (as pandas dataframes) 
X = communities_and_crime.data.features 
y = communities_and_crime.data.targets 
  
# metadata 
print(communities_and_crime.metadata) 
  
# variable information 
print(communities_and_crime.variables) 


{'uci_id': 183, 'name': 'Communities and Crime', 'repository_url': 'https://archive.ics.uci.edu/dataset/183/communities+and+crime', 'data_url': 'https://archive.ics.uci.edu/static/public/183/data.csv', 'abstract': 'Communities within the United States. The data combines socio-economic data from the 1990 US Census, law enforcement data from the 1990 US LEMAS survey, and crime data from the 1995 FBI UCR.', 'area': 'Social Science', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 1994, 'num_features': 127, 'feature_types': ['Real'], 'demographics': ['Race', 'Age', 'Income', 'Occupation'], 'target_col': ['ViolentCrimesPerPop'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2002, 'last_updated': 'Mon Mar 04 2024', 'dataset_doi': '10.24432/C53W3X', 'creators': ['Michael Redmond'], 'intro_paper': {'ID': 405, 'type': 'NATIVE', 'title': 'A data-driven software tool for enabling cooperative information s

In [4]:
X.head()

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.29,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,?,0.02,0.12,0.45,?,?,?,?,0.00,?
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,?,0.01,0.21,0.02,?,?,?,?,0.00,?
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,?,0.02,0.39,0.28,?,?,?,?,0.00,?
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,?,0.04,0.09,0.02,?,?,?,?,0.00,?


In [5]:
X = X.drop(['state', 'county', 'community', 'communityname', 'fold'], axis=1)
X.head()

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.29,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,?,0.02,0.12,0.45,?,?,?,?,0.00,?
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,?,0.01,0.21,0.02,?,?,?,?,0.00,?
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,?,0.02,0.39,0.28,?,?,?,?,0.00,?
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,?,0.04,0.09,0.02,?,?,?,?,0.00,?


In [6]:
y.head()

,ViolentCrimesPerPop
0,0.20
1,0.67
2,0.43
3,0.12
4,0.03


In [7]:
X.replace('?', np.nan, inplace=True)

null_counts = X.isnull().sum()

# Filter columns with more than 0 nulls
null_counts = null_counts[null_counts > 0]

# Print the result
print(null_counts)

OtherPerCap                1
LemasSwornFT            1675
LemasSwFTPerPop         1675
LemasSwFTFieldOps       1675
LemasSwFTFieldPerPop    1675
LemasTotalReq           1675
LemasTotReqPerPop       1675
PolicReqPerOffic        1675
PolicPerPop             1675
RacialMatchCommPol      1675
PctPolicWhite           1675
PctPolicBlack           1675
PctPolicHisp            1675
PctPolicAsian           1675
PctPolicMinor           1675
OfficAssgnDrugUnits     1675
NumKindsDrugsSeiz       1675
PolicAveOTWorked        1675
PolicCars               1675
PolicOperBudg           1675
LemasPctPolicOnPatr     1675
LemasGangUnitDeploy     1675
PolicBudgPerPop         1675
dtype: int64


In [8]:
# Identify columns where conversion to numeric fails
for col in X.columns:
    try:
        X[col] = pd.to_numeric(X[col], errors='raise')
    except ValueError:
        print(f"Column '{col}' contains non-numeric values")


In [9]:
X.dtypes[X.dtypes != np.float64]
# Good, everything has been converted to numerical !
# Now we can deal with Null values

Series([], dtype: object)

In [10]:
X['OtherPerCap'] = X['OtherPerCap'].fillna(X['OtherPerCap'].mean()) # Since there is 1 value missing, we replace with the mean

null_counts.drop('OtherPerCap', inplace=True) # We drop the 'OtherPerCap' from null_count table since this column has no missing values anymore
X.drop(null_counts.index.to_list(), axis=1, inplace=True) # For other, since there is too many missing, values, we drop this features from X
X.info

<bound method DataFrame.info of       population  householdsize  racepctblack  racePctWhite  racePctAsian  \
0           0.19           0.33          0.02          0.90          0.12   
1           0.00           0.16          0.12          0.74          0.45   
2           0.00           0.42          0.49          0.56          0.17   
3           0.04           0.77          1.00          0.08          0.12   
4           0.01           0.55          0.02          0.95          0.09   
...          ...            ...           ...           ...           ...   
1989        0.01           0.40          0.10          0.87          0.12   
1990        0.05           0.96          0.46          0.28          0.83   
1991        0.16           0.37          0.25          0.69          0.04   
1992        0.08           0.51          0.06          0.87          0.22   
1993        0.20           0.78          0.14          0.46          0.24   

      racePctHisp  agePct12t21  agePct12t29

# Implementation
Now that the preprocessing is done, having a X and y clean, let's implements the assigmenet methods.

In [ ]:
# Function to check if the method given to compute the score  exists: 
score_methods_array = ['RMSE', ]

def method_error(score_used):
    return 0

## Cross Validation and leave-one-out method
### Cross Validation

In [52]:
# First let's code the function eval_MSE used in ISLP to evalutate the MSE given the features we want to include in the model
def evalMSE_for_linear_regression(terms, X_train, y_train, X_test, y_test):
    '''
    Calculate the Mean Squared Error (MSE) between predicted and actual values.

    Parameters:
    ----------
    terms : array-like, shape (n_selected_features,)
        The predictor variables (or features) used in the model.
        
    X_train : array-like, shape (n_samples_train, n_features)
        Training data from the original preprocessed dataFrame (without the target column)
        
    y_test : array-like, shape (n_samples_train,)
        The target value column of the original preprocessed dataFrame used for training.

    X_test : array-like, shape (n_samples_test, n_features)
        Testing data from the original preprocessed dataFrame (without the target column)
        
    y_test : array-like, shape (n_samples_test,)
        The target value column of the original preprocessed dataFrame used for testing.

    Returns:
    --------
    mse : float
        The calculated mean squared error on the test data.
    '''

    #intercept_train = pd.DataFrame({'intercept' : np.ones(X_train.shape[0])}, index=X_train.index)
    #X_train = pd.concat([intercept_train, X_train[terms]], axis=1) # Construction of the X_train transform to use the linear regression of sklearn or statsmodels

    #intercept_test = pd.DataFrame({'intercept' : np.ones(X_train.shape[0])}, index=X_train.index)
    # X_test = pd.concat([intercept_test, X_test[terms]], axis=1)

    # I wanted to do it by hand but the .predict function needs .fit_transform and .transform specifically.

    mm = MS(terms)

    X_train = mm.fit_transform(X_train)
    X_test = mm.transform(X_test)

    y_train = y_train['ViolentCrimesPerPop']
    y_test = y_test['ViolentCrimesPerPop']

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)

    return np.mean((y_test - test_pred)**2)

In [47]:
def k_fold_CV(X, y, terms, k=5, tqdm_disable="False"):
    '''
    Calculate the MSE using k-fold Cross Validation method.

    Parameters:
    ----------
    X : array-like, shape (n_samples, n_features)
        Data from the original preprocessed dataFrame (without the target column)
        
    y : array-like, shape (n_samples,)
        The target value column of the original preprocessed dataFrame.

    terms : array-like, shape (n_selected_features,)
        The predictor variables (or features) used in the model.
        
    k : integer
        number of cross validation

    tqdm_disable : Bool
        Enable it when it is too heavy to watch the process of the loops, especially when this function is called numerous times

    Returns:
    --------
    mse : float
        The mean of different calculated mean squared error on the test data for different folds.
    '''

    # check if k is valid
    if k > X.shape[0]:
        raise ValueError(f"k cannot be greater than the number of samples. k={k}, but X contains only {X.shape[0]} samples.")

    k_MSE = np.zeros(k)

    X_shuffled, y_shuffled = shuffle(X, y, random_state=1)

    X_parts = np.array_split(X_shuffled, k)
    y_parts = np.array_split(y_shuffled, k)

    for i in tqdm(range(k), disable=tqdm_disable):
        X_test = X_parts[i]
        y_test = y_parts[i]

        # X_train = np.concatenate([part for j,part in enumerate(X_parts) if j != i])
        # y_train = np.concatenate([part for j,part in enumerate(y_parts) if j != i]) 
        X_train = pd.concat([part for j,part in enumerate(X_parts) if j != i], axis=0)
        y_train = pd.concat([part for j,part in enumerate(y_parts) if j != i], axis=0) 
        # [part for j,part in enumerate(X_parts) if j != i] generates an array of arrays of X_parts without the i-th part
        # then it concatenates all the given arrays into one array X_train

        k_MSE[i] = evalMSE_for_linear_regression(terms, X_train, y_train, X_test, y_test)
    
    return k_MSE.mean()

In [26]:
MSE_k5_Cross_Validation = k_fold_CV(X=X, y=y, terms=['population', 'householdsize', 'racepctblack', 'racePctWhite'], k=5)

c:\Users\doria\Documents\pythonEnv\dataScience\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 16.20it/s]


In [27]:
MSE_k5_Cross_Validation

0.0252765560842217

### Leave-one-out Cross Validation (LOOCV)
I'm the function `k_fold_CV` with $k$ = to the number of datapoints of the preprocessed dataset `X` (lets say $N_X$). The function then divides `X` and `y` into $N_X$ folds. But in this case $N_X$ folds means that one fold is equal to one value of the dataset `X`,`y`. So during one iteration `i`of the `for` loop, it trains over the whole dataset except the `i`th value, and this for the whole dataset.

In [ ]:
MSE_LOOCV = k_fold_CV(X=X, y=y, terms=['population', 'householdsize', 'racepctblack', 'racePctWhite'], k=X.shape[0]) 
# if k > length of the dataset, an error is raised


In [29]:
MSE_LOOCV

0.02535316426092495

## Foward attribute selection

To do:

- Create a loop on `terms` to compute MSE using `k_fold_CV`
- On each turn, saving the term/combination of terms that gives the best MSE.
- then create a while loop (which includes the previous loop) that prevents overfitting by checking the MSE
- extend the function to other score

In [48]:
def foward_attribute_selection(X, y, k=5, score_used='MSE'):
    '''
    Returns the selected attributes that works best for training the model without overfitting. 
    Usage of foward attribute selection method.

    Parameters:
    ----------
    X : array-like, shape (n_samples, n_features)
        Data from the original preprocessed dataFrame (without the target column)
        
    y : array-like, shape (n_samples,)
        The target value column of the original preprocessed dataFrame.
        
    k : integer
        number of cross validation
    
    score_used : char
        usage of score (R2, MSE, etc)

    Returns:
    --------
    selected_terms : array-like, variable shape
        Selected attributes that works best for training the model without overfitting.
    '''

    all_terms = X.columns.values

    
    
    if score_used == 'MSE':
        lim_score = np.inf
        compare = lambda a, b: a < b
    elif score_used == 'R2':
        lim_score = -np.inf
        compare = lambda a, b: a > b
    else:
        raise ValueError(f"Score {score_used} is not supported or does not exists. Choose your scores between `MSE` or `R2`.")
        
    best_global_score = lim_score
    best_current_score = lim_score

    best_current_terms = np.array([])
    best_global_terms = np.array([]) 
    # To implement backward aswell, do a if function as I did for the score_used and fill or not these two

    while True:
        best_current_score = lim_score

        for term in all_terms:
            testing_terms = np.hstack([best_global_terms, [term]])
            score = k_fold_CV(X=X, y=y, terms=testing_terms, tqdm_disable=True)

            if compare(score, best_current_score):
                best_current_terms = testing_terms
                best_current_score = score

        print(f"New term added: {best_current_terms[-1]} to {best_global_terms}.")
        print(f"{score_used} for theses terms is: {best_current_score}.")
        if compare(best_current_score, best_global_score):
            print(f"It is a better score than {best_global_score}. New selected terms is {best_current_terms}")
            best_global_terms = best_current_terms
            best_global_score = best_current_score
        else: # maybe add a count, like 2 or 3 to to another loop to check if we are overfitting or if we may have another ehancement.
            print(f"OVERFITTING: Score does not imporve compare to the previous score {best_global_score}.\n")
            print(f"Final selected terms for training is {best_current_terms}")
            return(best_global_terms)


In [51]:
warnings.filterwarnings("ignore") # disabling warnings for a clearer view
foward_attribute_selection(X=X,y=y)
warnings.filterwarnings("default")

New term added: PctKids2Par to [].
MSE for theses terms is: 0.024730824263345174.
It is a better score than inf. New selected terms is ['PctKids2Par']
New term added: racePctWhite to ['PctKids2Par'].
MSE for theses terms is: 0.021820891012692772.
It is a better score than 0.024730824263345174. New selected terms is ['PctKids2Par' 'racePctWhite']
New term added: HousVacant to ['PctKids2Par' 'racePctWhite'].
MSE for theses terms is: 0.02035807509749834.
It is a better score than 0.021820891012692772. New selected terms is ['PctKids2Par' 'racePctWhite' 'HousVacant']
New term added: pctUrban to ['PctKids2Par' 'racePctWhite' 'HousVacant'].
MSE for theses terms is: 0.020027668209430183.
It is a better score than 0.02035807509749834. New selected terms is ['PctKids2Par' 'racePctWhite' 'HousVacant' 'pctUrban']
New term added: PctWorkMom to ['PctKids2Par' 'racePctWhite' 'HousVacant' 'pctUrban'].
MSE for theses terms is: 0.019763471413352125.
It is a better score than 0.020027668209430183. New s

KeyboardInterrupt: 